In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('dataset2.xls')
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [4]:
df.Type.nunique()

6

In [5]:
X = df.drop(['Type'], axis = 1)
y = df['Type']

Итак, мы выяснили, сколько у нас классов и узнали, что в этот раз у нас будет не бинарная классификация. А значит, мы не сможем использовать некоторые метрики качества (например, preccision и recall).

Приступим непосредственно к построению модели. На примере этой задачи мы узнаем новый алгоритм kNN (метод ближайших соседей). Это один из простейших методов классификации. 

Его называют ленивым классификатором, потому что во время обучения модели он ничего не делает, просто считывает и сохраняет тренировочные данные. Сама классификация для него начинается тогда, когда ему дают тестовые данные. Тогда kNN проходит два базовых шага:

Сначала он ищет k ближайших размеченных точек данных – эти точки и называют k ближайшими соседями.
Затем, используя классы соседей, kNN решает, как лучше классифицировать новые данные. 

Пример

Предположим, что алгоритм хочет классифицировать вас по профессии. Он видит, что люди, рядом с которыми вы находитесь (родители, друзья), — врачи. Тогда он решает, что вы — тоже скорее всего врач. Грубо говоря, он основывается на принципе, что подобное находится рядом с подобным.

Итак, мы разобрались с новым алгоритмом. Теперь воспользуемся k-fold валидацией на пяти разбиениях и обучим модель:

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold
model=KNeighborsClassifier(n_neighbors=3)
kf = KFold(n_splits=10)
acc = cross_val_score(model, X, y, cv=kf, scoring="accuracy")
acc

array([0.45454545, 0.77272727, 0.59090909, 0.77272727, 0.28571429,
       0.61904762, 0.42857143, 0.14285714, 0.23809524, 0.95238095])

In [7]:
np.mean(acc)

0.5257575757575758

В следующей задаче мы будем диагностировать болезни сердца по различным медицинским параметрам пациентов.

In [8]:
df = pd.read_csv('heart.csv', sep = ';')
df.head()

,age,sex,cp,trestbps,chol,restecg,thalach,exang,oldpeak,target
0,63,1,3,145,233,0,150,0,2.3,1
1,37,1,2,130,250,1,187,0,3.5,1
2,41,0,1,130,204,0,172,0,1.4,1
3,56,1,1,120,236,1,178,0,0.8,1
4,57,0,0,120,354,1,163,1,0.6,1


In [9]:
df.describe()

,age,sex,cp,trestbps,chol,restecg,thalach,exang,oldpeak,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.528053,149.646865,0.326733,1.039604,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.525860,22.905161,0.469794,1.161075,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,71.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,133.500000,0.000000,0.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,1.000000,153.000000,0.000000,0.800000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,1.000000,166.000000,1.000000,1.600000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,2.000000,202.000000,1.000000,6.200000,1.000000


In [10]:
IQR = (df.quantile([0.75])).iloc[0] - (df.quantile([0.25])).iloc[0]

In [11]:
IQR

age         13.5
sex          1.0
cp           2.0
trestbps    20.0
chol        63.5
restecg      1.0
thalach     32.5
exang        1.0
oldpeak      1.6
target       1.0
dtype: float64

In [12]:
q75 =df.quantile([0.75]).iloc[0]
q25 = df.quantile([0.25]).iloc[0]

In [13]:
q = df['age'].quantile([0.75])
q.iloc[0]

61.0

In [14]:
IQR_l = q25 - 1.5*IQR
IQR_m = q75 + 1.5*IQR


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   restecg   303 non-null    int64  
 6   thalach   303 non-null    int64  
 7   exang     303 non-null    int64  
 8   oldpeak   303 non-null    float64
 9   target    303 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 23.8 KB


In [16]:
for i in df.columns:
    print(i)

age
sex
cp
trestbps
chol
restecg
thalach
exang
oldpeak
target


In [20]:
for i in df.columns:
    df[i] = df[(df[i]<IQR_m[i])&(df[i]>IQR_l[i])]

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  290 non-null    float64
 4   chol      298 non-null    float64
 5   restecg   303 non-null    float64
 6   thalach   302 non-null    float64
 7   exang     303 non-null    float64
 8   oldpeak   295 non-null    float64
 9   target    303 non-null    float64
dtypes: float64(10)
memory usage: 23.8 KB


In [23]:
df.isna().sum()

age          0
sex          0
cp           0
trestbps    13
chol         5
restecg      0
thalach      1
exang        0
oldpeak      8
target       0
dtype: int64

In [25]:
df = df.dropna(axis = 0)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279 entries, 0 to 302
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       279 non-null    float64
 1   sex       279 non-null    float64
 2   cp        279 non-null    float64
 3   trestbps  279 non-null    float64
 4   chol      279 non-null    float64
 5   restecg   279 non-null    float64
 6   thalach   279 non-null    float64
 7   exang     279 non-null    float64
 8   oldpeak   279 non-null    float64
 9   target    279 non-null    float64
dtypes: float64(10)
memory usage: 24.0 KB


In [27]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
restecg     0
thalach     0
exang       0
oldpeak     0
target      0
dtype: int64

In [29]:
col_list = df.columns

In [35]:
data = pd.read_csv('heart.csv',sep=';',encoding='cp1251')
def del_outliers(df, col_name):
    if df[col_name].dtype != 'object':
        col = df[col_name]
        quant_list = [col.quantile(0.25), col.quantile(0.75)]
        iqr = quant_list[1] - quant_list[0]
        diap_list = [quant_list[0] - 1.5*iqr, quant_list[1] + 1.5*iqr]
        diap_0, diap_1 = diap_list[0], diap_list[1]
        return df.query(f"{diap_0} < {col_name} < {diap_1}")
    else:
        return df
for col in col_list:
    data = del_outliers(data, col)
data.drop(columns=['oldpeak'], inplace=True)
display(data)
X, y = data.drop(columns=['target']).values, data['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=5)
model = LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:,1]
res = roc_auc_score(y_test, y_pred_proba)
display(type(res), res.round(3))

,age,sex,cp,trestbps,chol,restecg,thalach,exang,target
0,63,1,3,145,233,0,150,0,1
1,37,1,2,130,250,1,187,0,1
2,41,0,1,130,204,0,172,0,1
3,56,1,1,120,236,1,178,0,1
4,57,0,0,120,354,1,163,1,1
...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,1,123,1,0
299,45,1,3,110,264,1,132,0,0
300,68,1,0,144,193,1,141,0,0
301,57,1,0,130,131,1,115,1,0


numpy.float64

0.769

In [34]:
data.describe()

,age,sex,cp,trestbps,chol,restecg,thalach,exang,target
count,279.000000,279.000000,279.000000,279.000000,279.000000,279.00000,279.000000,279.000000,279.000000
mean,53.874552,0.698925,0.985663,129.301075,241.924731,0.53405,150.064516,0.311828,0.566308
std,9.194659,0.459550,1.021254,14.716104,44.645342,0.51393,22.875550,0.464072,0.496474
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.00000,88.000000,0.000000,0.000000
25%,46.000000,0.000000,0.000000,120.000000,209.500000,0.00000,132.500000,0.000000,0.000000
50%,54.000000,1.000000,1.000000,130.000000,239.000000,1.00000,153.000000,0.000000,1.000000
75%,60.000000,1.000000,2.000000,140.000000,269.500000,1.00000,168.000000,1.000000,1.000000
max,77.000000,1.000000,3.000000,165.000000,360.000000,2.00000,202.000000,1.000000,1.000000


In [37]:
data.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
restecg     0
thalach     0
exang       0
target      0
dtype: int64

In [39]:
133*10+133

1463